In [ ]:
!nvidia-smi

Thu Aug 24 23:24:43 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns


In [ ]:
df= pd.read_csv("car_acceptability.txt", header=None)

Column isimlerini uygun bir şekilde değiştirdim

In [ ]:
df.columns = ["alis", "parca", "kapi", "kapasite", "bagaj", "guvenlik", "alinabilirlik"]

Her kolonda hangi değerden kaçar tane olduğunu inceledim

In [ ]:
[df[col].value_counts() for col in df.columns]


[high     432
 med      431
 vhigh    430
 low      430
 ?          3
 -          1
 düsük      1
 Name: alis, dtype: int64,
 high     432
 med      432
 low      432
 vhigh    429
 ?          4
 Name: parca, dtype: int64,
 3        432
 2        430
 4        430
 5more    430
 ?          3
 -          1
 44         1
 iki        1
 5+         1
 Name: kapi, dtype: int64,
 4       576
 more    576
 2       574
 ?         3
 Name: kapasite, dtype: int64,
 small    576
 big      576
 med      575
 ?          2
 Name: bagaj, dtype: int64,
 high    576
 med     574
 low     573
 ?         2
 -         2
 *         1
 Name: guvenlik, dtype: int64,
 unacc    1209
 acc       384
 good       69
 vgood      65
 ?           2
 Name: alinabilirlik, dtype: int64]

Uygun olmayan değerleri tablodaki uygun olduğunu düşündüğüm değerlere dönüştürdüm

In [ ]:
df["kapi"].replace("5+", "5more", inplace=True)

In [ ]:
df["kapi"].replace("44", "4", inplace=True)

In [ ]:
df["kapi"].replace("iki", "2", inplace=True)

In [ ]:
df["alis"].replace("düsük", "low", inplace=True)


In [ ]:
df = df[~(df == "-").any(axis=1)]

In [ ]:
df = df[~(df == "?").any(axis=1)]
df = df[~(df == "*").any(axis=1)]

In [ ]:
[df[col].value_counts() for col in df.columns]


[high     432
 med      428
 low      428
 vhigh    427
 Name: alis, dtype: int64,
 low      432
 high     430
 med      430
 vhigh    424
 Name: parca, dtype: int64,
 3        432
 4        429
 5more    428
 2        427
 Name: kapi, dtype: int64,
 4       573
 more    573
 2       570
 Name: kapasite, dtype: int64,
 small    573
 big      572
 med      571
 Name: bagaj, dtype: int64,
 high    573
 med     572
 low     570
 Name: guvenlik, dtype: int64,
 unacc    1201
 acc       381
 good       69
 vgood      65
 Name: alinabilirlik, dtype: int64]

Dataframein bir kopyasını oluşturdum bir karışıklık olması durumunda dönüp bu kopyadan devam edebilmek için

In [ ]:
dff = df.copy()

In [ ]:
dff.value_counts()

alis   parca  kapi   kapasite  bagaj  guvenlik  alinabilirlik
high   high   2      2         big    high      unacc            1
med    med    4      more      small  high      acc              1
                               med    low       unacc            1
                                      high      vgood            1
                               big    med       acc              1
                                                                ..
low    low    3      2         big    med       unacc            1
                                      low       unacc            1
                                      high      unacc            1
              2      more      small  med       unacc            1
vhigh  vhigh  5more  more      small  med       unacc            1
Length: 1714, dtype: int64

In [ ]:
[dff[col].value_counts() for col in dff.columns]

[high     432
 med      428
 low      428
 vhigh    427
 Name: alis, dtype: int64,
 low      432
 high     430
 med      430
 vhigh    424
 Name: parca, dtype: int64,
 3        432
 4        429
 5more    428
 2        427
 Name: kapi, dtype: int64,
 4       573
 more    573
 2       570
 Name: kapasite, dtype: int64,
 small    573
 big      572
 med      571
 Name: bagaj, dtype: int64,
 high    573
 med     572
 low     570
 Name: guvenlik, dtype: int64,
 unacc    1201
 acc       381
 good       69
 vgood      65
 Name: alinabilirlik, dtype: int64]

In [ ]:
pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 MB 9.7 MB/s eta 0:00:00


Gerekli import işlemlerini yaptım

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from catboost import CatBoostClassifier, Pool

In [ ]:
X, y = dff.drop(["alinabilirlik"], axis=1), dff["alinabilirlik"]

Modelimi kurmak için train ve test olarak datamı böldüm

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size = 0.2,
    stratify=y
)

Daha önce farklı bir ortamda kurmuş olduğum modelle bulduğum en iyi değerleri veren parametrelerimi ekledim

In [ ]:
model_results = CatBoostClassifier(depth= 5, l2_leaf_reg= 1, learning_rate= 0.3)

In [ ]:
categorical_features = list(X_train.select_dtypes(include = "object").columns)
categorical_features

['alis', 'parca', 'kapi', 'kapasite', 'bagaj', 'guvenlik']

Modelimi kurup çalıştırdıktan sonra nan değerlerin olduğu için modelim çalışmadı mevcut nan değerleri inceledim. Göz ardı edilebilecek sayıdaydı 1-2 gibi. Onları dönüp drop ettim

In [ ]:
dff.dropna(inplace=True)

In [ ]:
dff.isnull().sum()

alis             0
parca            0
kapi             0
kapasite         0
bagaj            0
guvenlik         0
alinabilirlik    0
dtype: int64

In [ ]:
model_results.fit(X_train,y_train, cat_features = categorical_features)

0:	learn: 1.0264066	total: 50.7ms	remaining: 50.7s
1:	learn: 0.8182542	total: 60.1ms	remaining: 30s
2:	learn: 0.7092862	total: 67ms	remaining: 22.3s
3:	learn: 0.6443080	total: 75.5ms	remaining: 18.8s
4:	learn: 0.5989683	total: 86ms	remaining: 17.1s
5:	learn: 0.5244729	total: 99.2ms	remaining: 16.4s
6:	learn: 0.4784327	total: 110ms	remaining: 15.7s
7:	learn: 0.4422055	total: 123ms	remaining: 15.2s
8:	learn: 0.4089736	total: 135ms	remaining: 14.8s
9:	learn: 0.3389696	total: 146ms	remaining: 14.5s
10:	learn: 0.3078009	total: 158ms	remaining: 14.2s
11:	learn: 0.2826498	total: 170ms	remaining: 14s
12:	learn: 0.2578201	total: 181ms	remaining: 13.7s
13:	learn: 0.2457415	total: 190ms	remaining: 13.4s
14:	learn: 0.2427851	total: 201ms	remaining: 13.2s
15:	learn: 0.2271857	total: 214ms	remaining: 13.1s
16:	learn: 0.2155954	total: 225ms	remaining: 13s
17:	learn: 0.2030664	total: 236ms	remaining: 12.9s
18:	learn: 0.2015016	total: 246ms	remaining: 12.7s
19:	learn: 0.1997941	total: 264ms	remaining: 

In [ ]:
model_preds = model_results.predict(X_test)

In [ ]:
print(classification_report(y_train, model_results.predict(X_train), digits = 3))

              precision    recall  f1-score   support

         acc      0.987     0.987     0.987       305
        good      0.902     1.000     0.948        55
       unacc      1.000     0.994     0.997       959
       vgood      1.000     1.000     1.000        52

    accuracy                          0.993      1371
   macro avg      0.972     0.995     0.983      1371
weighted avg      0.993     0.993     0.993      1371



In [ ]:
print(classification_report(y_test, model_preds, digits = 3))

              precision    recall  f1-score   support

         acc      0.935     0.947     0.941        76
        good      0.737     1.000     0.848        14
       unacc      1.000     0.975     0.987       240
       vgood      1.000     1.000     1.000        13

    accuracy                          0.971       343
   macro avg      0.918     0.981     0.944       343
weighted avg      0.975     0.971     0.972       343



In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

Aldığım skoru inceledim gayet yüksek olduğunu gördüm

In [ ]:
accuracy_score(y_test, model_preds)

0.9708454810495627

In [ ]:
conf_matrix = confusion_matrix(y_test, model_preds)

In [ ]:
print(conf_matrix)

[[ 72   4   0   0]
 [  0  14   0   0]
 [  5   1 234   0]
 [  0   0   0  13]]


Proje bitti
